# Tratamiento de datos nulos

La diferencia entre los datos que se encuentran en muchos tutoriales y los datos del mundo real es que los datos del mundo real rara vez son limpios y homogéneos.
En particular, **muchos conjuntos de datos interesantes tendrán alguna cantidad de datos faltantes**.
Para complicar aún más las cosas, diferentes fuentes de datos pueden indicar los datos que faltan de diferentes maneras.

En esta sección, discutiremos algunas consideraciones generales para los datos perdidos, discutiremos cómo Pandas elige representarlos, y demostraremos algunas herramientas incorporadas de Pandas para manejar los datos faltantes (missings) en Python.
**Nos referiremos a los datos perdidos en general como valores *nulos*, *missing*, *NaN* o *NA* .



## "Compromisos" en las convenciones de datos faltantes


Hay una serie de esquemas que se han desarrollado para indicar la presencia de datos que faltan en una tabla o DataFrame.
Generalmente, giran en torno a una de dos estrategias: **utilizar una *máscara* que indique globalmente los valores que faltan, o elegir un *valor centinela* que indique una entrada que falta**.

En el enfoque de enmascaramiento, **la máscara** puede ser una matriz booleana completamente separada, o puede implicar la apropiación de un bit en la representación de datos para indicar localmente el estado nulo de un valor.

En el enfoque de centinela, el valor centinela **puede ser alguna convención específica de los datos, como indicar un valor entero faltante con -9999** o algún patrón de bits raro, o puede ser una convención más global, como indicar un valor de punto flotante faltante con **NaN (Not a Number)**, un valor especial que forma parte de la especificación de punto flotante del IEEE.

Ninguno de estos enfoques está exento de inconvenientes: el uso de una matriz de máscaras separada requiere la asignación de una matriz booleana adicional, lo que añade una sobrecarga tanto de almacenamiento como de cálculo. Un valor centinela reduce el rango de valores válidos que pueden representarse y puede requerir una lógica adicional (a menudo no optimizada) en la aritmética de la CPU y la GPU. Los valores especiales comunes como NaN no están disponibles para todos los tipos de datos.

Como en la mayoría de los casos en los que no existe una opción universalmente óptima, diferentes lenguajes y sistemas utilizan diferentes convenciones.
**Por ejemplo, el lenguaje R utiliza patrones de bits reservados dentro de cada tipo de datos como valores centinela que indican la ausencia de datos**, mientras que el sistema SciDB utiliza un byte adicional adjunto a cada celda que indica un estado NA.

## Missing Data en Pandas

La forma en que Pandas **maneja los valores perdidos está limitada por su dependencia del paquete NumPy**, que no tiene una noción incorporada de valores NA para los tipos de datos que no son de punto flotante.

Pandas podría haber seguido el ejemplo de **R especificando patrones de bits para cada tipo de datos individual** para indicar la nulidad, pero este enfoque resulta ser bastante difícil de manejar.
Mientras que R contiene cuatro tipos de datos básicos, **NumPy soporta *mucho* más que esto**: por ejemplo, mientras que R tiene un único tipo de entero, NumPy soporta *catorce* tipos de enteros básicos una vez que se tienen en cuenta las precisiones disponibles, la significación y la codificación.
**Reservar un patrón de bits específico en todos los tipos disponibles de NumPy llevaría a una cantidad inmanejable de sobrecarga** en las operaciones especiales para varios tipos, probablemente incluso requiriendo un nuevo fork del paquete NumPy. Además, para los tipos de datos más pequeños (como los enteros de 8 bits), sacrificar un bit para usarlo como máscara reducirá significativamente el rango de valores que puede representar.

**NumPy tiene soporte para arrays enmascarados** - es decir, arrays que tienen un array de máscara booleana separado para marcar los datos como "buenos" o "malos".
Pandas podría haber derivado de esto, pero la sobrecarga en el almacenamiento, el cálculo y el mantenimiento del código hace que sea una opción poco atractiva.

Con estas limitaciones en mente, **Pandas eligió usar centinelas para los datos perdidos**, y además eligió usar dos valores nulos ya existentes en Python: el valor especial de punto flotante **``NaN``, y el objeto Python ``None``**.
Esta elección tiene algunos efectos secundarios, como veremos, pero en la práctica acaba siendo un buen compromiso en la mayoría de los casos de interés.

### ``None``: Pythonic missing data

El primer valor centinela utilizado por Pandas es **``None``**, un objeto singleton de Python que se utiliza a menudo para los datos que faltan en el código de Python.
Debido a que es un objeto de Python, ``None`` no se puede utilizar en cualquier array arbitrario de NumPy/Pandas, sino sólo en arrays con tipo de datos ``'object'`` (es decir, arrays de objetos de Python):

In [1]:
import numpy as np
import pandas as pd

In [2]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

Este ``dtype=object`` significa que la mejor representación de tipo común que NumPy puede inferir para los contenidos del array es que son objetos de Python.
Aunque este tipo de matriz de objetos es útil para algunos propósitos, cualquier operación sobre los datos se hará a nivel de Python, con mucha más sobrecarga que las operaciones típicamente rápidas vistas para las matrices con tipos nativos:

In [3]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
65 ms ± 3.26 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
2.3 ms ± 180 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



El uso de objetos Python en un array también significa que si realizas agregaciones como ``sum()`` o ``min()`` en un array con un valor ``None``, generalmente obtendrás un error:

In [4]:
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

In [5]:
np.nansum(vals1)

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

Esto refleja el hecho de que no está definida la suma entre los números enteros y los de tipo `None`.

### ``NaN``: Missing datos numéricos

La otra representación de datos faltantes, ``NaN`` (acrónimo de *Not a Number*), es diferente; es un valor especial de punto flotante reconocido por todos los sistemas que utilizan la representación estándar de punto flotante IEEE:

In [7]:
vals2 = np.array([1, np.nan, 3, 4]) 
vals2.dtype

dtype('float64')

Fíjate en que NumPy eligió un tipo nativo de punto flotante para este array: esto significa que **a diferencia del array de objetos de antes, este array soporta operaciones rápidas empujadas en el código compilado.**
Debes tener en cuenta que ``NaN`` es un poco como un virus de datos - infecta cualquier otro objeto que toque.
Independientemente de la operación, el resultado de la aritmética con ``NaN`` será otro ``NaN``:

In [8]:
1 + np.nan

nan

In [9]:
0 *  np.nan

nan

In [10]:
vals2

array([ 1., nan,  3.,  4.])

Tenga en cuenta que esto significa que los agregados sobre los valores están bien definidos (es decir, no dan lugar a un error) pero no siempre son útiles:

In [11]:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

NumPy proporciona algunas agregaciones especiales que ignoran estos valores perdidos:

In [12]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

In [13]:
np.nanprod(vals2)

12.0

In [14]:
np.nanmean(vals2)

2.6666666666666665

In [23]:
(1 + 3 + 4) / 3

2.6666666666666665

Tenga en cuenta que ``NaN`` es específicamente un valor de punto flotante; no hay un valor NaN equivalente para enteros, cadenas u otros tipos.

### NaN y None en Pandas

Tanto ``NaN`` como ``None`` tienen su lugar, y Pandas está construido para manejar los dos casi indistintamente, convirtiendo entre ellos cuando sea apropiado:

In [16]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

Para los tipos que no tienen un valor centinela disponible, Pandas realiza automáticamente una conversión de tipo cuando hay valores NA.
Por ejemplo, **si establecemos un valor en un array de enteros a ``np.nan``, se convertirá automáticamente a un tipo de punto flotante para acomodar el NA**:

In [18]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int32

In [19]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

Observe que **además de convertir el array de enteros a punto flotante, Pandas convierte automáticamente el ``None`` a un valor ``NaN``**.
(Tenga en cuenta que hay una propuesta para añadir un entero nativo NA a Pandas en el futuro; en el momento de escribir esto, no se ha incluido).

Mientras que este tipo de magia puede parecer un poco deficiente en comparación con el enfoque más unificado de los valores NA en lenguajes específicos como R, el enfoque de centinela/casting de Pandas funciona bastante bien en la práctica y en mi experiencia sólo causa problemas en raras ocasiones.

La siguiente tabla lista las convenciones de upcasting en Pandas cuando se introducen valores NA:

|Typeclass     | Conversion When Storing NAs | NA Sentinel Value      |
|--------------|-----------------------------|------------------------|
| ``floating`` | No change                   | ``np.nan``             |
| ``object``   | No change                   | ``None`` or ``np.nan`` |
| ``integer``  | Cast to ``float64``         | ``np.nan``             |
| ``boolean``  | Cast to ``object``          | ``None`` or ``np.nan`` |

__Tenga en cuenta que en Pandas, los datos de cadena siempre se almacenan con un dtype ``object``.__

## Operar con valores nulos

Como hemos visto, **Pandas trata ``None`` y ``NaN`` como esencialmente intercambiables para indicar valores nulos o perdidos.**
Para facilitar esta convención, existen varios métodos útiles para detectar, eliminar y reemplazar valores nulos en las estructuras de datos de Pandas.
Estos son:

- ``isnull()``: Generar una máscara booleana que indique los valores que faltan
- ``notnull()``: El contrario de ``isnull()``
- ``dropna()``: Devuelve una versión filtrada de los datos
- ``fillna()``: Devuelve una copia de los datos con los valores perdidos rellenados o imputados

Veremos unos ejemplos de estas funciones.

### Detectar valores null

Las estructuras de datos de Pandas tienen dos métodos útiles para detectar datos nulos: ``isnull()`` y ``notnull()``.
Cualquiera de ellos devolverá una máscara booleana sobre los datos. Por ejemplo:

In [20]:
data = pd.Series([1, np.nan, 'hello', None])
data

0        1
1      NaN
2    hello
3     None
dtype: object

In [22]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [23]:
data.notnull()

0     True
1    False
2     True
3    False
dtype: bool

 Las máscaras booleanas se pueden utilizar directamente como índice de ``Series`` o ``DataFrame``:

In [27]:
data[data.notnull()]

0        1
2    hello
dtype: object

In [28]:
data[~(data==1)]

1      NaN
2    hello
3     None
dtype: object

In [30]:
data[~data.isnull()]

0        1
2    hello
dtype: object

Los métodos ``isnull()`` y ``notnull()`` producen resultados booleanos similares para los ``DataFrame``.

### Eliminando valores nulos

Además del enmascaramiento utilizado anteriormente, existen los métodos de conveniencia, ``dropna()``
(que elimina los valores NA) y ``fillna()`` (que rellena los valores NA). Para una ``Serie``,
el resultado es sencillo:

In [31]:
data

0        1
1      NaN
2    hello
3     None
dtype: object

In [36]:
# data.dropna()
data.dropna(inplace=True)
# data = data.dropna()

In [37]:
data

0        1
2    hello
dtype: object

Para un ``DataFrame``, hay más opciones.
Considera el siguiente ``DataFrame``:

In [38]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       2 non-null      float64
 1   1       2 non-null      float64
 2   2       3 non-null      int64  
dtypes: float64(2), int64(1)
memory usage: 204.0 bytes


No podemos eliminar valores individuales de un ``DataFrame``; sólo podemos eliminar filas o columnas completas.
Dependiendo de la aplicación, se puede querer una cosa u otra, por lo que ``dropna()`` da una serie de opciones para un ``DataFrame``.

Por defecto, ``dropna()`` eliminará todas las __filas__ en las que esté presente *cualquier* valor nulo:

In [40]:
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [42]:
df.dropna()

,0,1,2
1,2.0,3.0,5


Alternativamente, puede dejar los valores NA a lo largo de un eje diferente; ``axis=1`` deja todas las columnas que contienen un valor nulo:

In [43]:
df.dropna(axis=1)
# df.dropna(axis='columns')

,2
0,2
1,5
2,6


Pero esto deja caer algunos datos buenos también; usted podría estar más interesado en dejar caer filas o columnas con *todos* los valores NA, o una mayoría de valores NA.
Esto se puede especificar a través de los parámetros ``how`` o ``thresh``, que permiten un control fino del número de nulos a permitir.

El valor por defecto es ``how='any'``, de forma que cualquier fila o columna (dependiendo de la palabra clave ``axis``) que contenga un valor nulo será descartada.
También se puede especificar ``how='all'``, que sólo eliminará las filas/columnas que sean *todos* valores nulos:

In [44]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [46]:
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


Para un control más preciso, el parámetro ``thresh`` permite especificar un número mínimo de valores no nulos para la fila/columna que debe conservarse:

In [53]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [50]:
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


In [51]:
len(df.columns)*0.75

3.0

In [52]:
df.dropna(axis='rows', thresh=len(df.columns)*0.75)

,0,1,2,3
1,2.0,3.0,5,NaN


In [53]:
df.dropna(axis='columns', thresh=len(df)*0.75)

,2
0,2
1,5
2,6


Aquí se han eliminado la primera y la última fila, porque sólo contienen dos valores no nulos.

### Rellenar valores nulos

A veces, en lugar de eliminar los valores NA, se prefiere sustituirlos por un valor válido.
Este valor podría ser un número único como el cero, o podría ser algún tipo de imputación o interpolación de los valores buenos.
Se podría hacer esto en el lugar usando el método ``isnull()`` como máscara, pero como es una operación tan común Pandas proporciona el método ``fillna()``, que devuelve una copia del array con los valores nulos reemplazados.

Considera la siguiente ``Serie``:

In [55]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

Podemos rellenar las entradas NA con un único valor, como por ejemplo el cero:

In [56]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [57]:
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

Podemos especificar un forward-fill para propagar el valor anterior hacia adelante:

In [58]:
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [59]:
# forward-fill
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

O podemos especificar un relleno para propagar los siguientes valores hacia atrás:

In [60]:
# back-fill
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

Para los ``DataFrame``, las opciones son similares, pero también podemos especificar un ``eje`` a lo largo del cual tienen lugar los rellenos:

In [61]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [62]:
df[1]

0    NaN
1    3.0
2    4.0
Name: 1, dtype: float64

In [63]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [64]:
df[1].mean()

3.5

In [65]:
df.fillna(df[1].mean())

,0,1,2,3
0,1.0,3.5,2,3.5
1,2.0,3.0,5,3.5
2,3.5,4.0,6,3.5


In [66]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [68]:
# df[0].fillna(df[0].mean())
df[1].fillna(df[1].mean())

0    3.5
1    3.0
2    4.0
Name: 1, dtype: float64

In [69]:
df[4] = [np.nan,1,2]
df

,0,1,2,3,4
0,1.0,NaN,2,NaN,NaN
1,2.0,3.0,5,NaN,1.0
2,NaN,4.0,6,NaN,2.0


In [70]:
df[5] = [1,1,2]
df

,0,1,2,3,4,5
0,1.0,NaN,2,NaN,NaN,1
1,2.0,3.0,5,NaN,1.0,1
2,NaN,4.0,6,NaN,2.0,2


In [75]:
df.fillna(method='bfill',axis=1)

,0,1,2,3,4,5
0,1.0,2.0,2.0,1.0,1.0,1.0
1,2.0,3.0,5.0,1.0,1.0,1.0
2,4.0,4.0,6.0,2.0,2.0,2.0


Observe que si un valor anterior no está disponible durante un llenado hacia adelante, el valor NA permanece.